In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from numpy import set_printoptions
import plotly.graph_objects as go
import joblib

### Data cleaning

In [ ]:
#DATA FROM COSMED
os.chdir(r"COSMED_FILEPATH")

ee = pd.read_excel("cosmed_filename.xlsx", usecols=[3,4,9,10,13,14,61,65,67,69,73], names=["label","data","time","RR","VO2","VCO2","TDEE","PRO","FAT","CHO","npRQ"])

date = ee["data"][0]
time = "08:27:32"
date_time_str = date+" "+time
ee.drop([0, 1], inplace=True)
ee.drop(columns=['label', 'data'], inplace=True)
start_time = datetime.strptime(date_time_str, '%d/%m/%Y %H:%M:%S')
ee["timestep"] = 0
for i in ee.index:
    ee["timestep"][i] =  start_time + timedelta(hours=int(ee["time"][i][:2]), minutes=int(ee["time"][i][3:5]), seconds=int(ee["time"][i][6:]))

ee.drop(columns=['time'], inplace=True)
ee['timestep'] = pd.to_datetime(ee['timestep'],format="%Y/%m/%d %H:%M:%S")
ee['TDEE_avg'] = ee['TDEE'].rolling(15).mean()
ee = ee.sort_values(by='timestep', ascending=True)

#DATA FROM ENVIRONMENTAL MEASUREMENTS
os.chdir(r"ENV_filepath")

a3 = pd.read_csv("A3.csv", skiprows=2, usecols=[1,2,3,4,5], names=["timestep","Tair_a3","Tgl_a3","Vair_a3","Tair,v_a3"])
a3['timestep'] = pd.to_datetime(a3['timestep'])
a3 = a3.sort_values(by='timestep', ascending=True)

b3 = pd.read_csv("B3.csv", skiprows=2, usecols=[1,2,3,4,5], names=["timestep","Tair_b3","Tgl_b3","Vair_b3","Tair,v_b3"])
b3['timestep'] = pd.to_datetime(b3['timestep'])
b3 = b3.sort_values(by='timestep', ascending=True)


#DATA FROM CORE
os.chdir(r"CORE_FILEPATH")

core = pd.read_excel("CORE_filename.xlsx", usecols=[0,7,8,9,12], names=["timestep","ACCx_lchest","ACCy_lchest","ACCz_lchest","Tcore_lchest"], skiprows=14)
core['timestep'] = pd.to_datetime(core['timestep'])
core["ACCmag_lchest"] = np.sqrt(core["ACCx_lchest"]**2 + core["ACCy_lchest"]**2 + core["ACCz_lchest"]**2)
core['timestep'] = core['timestep'].dt.floor('s')   
core = core.sort_values(by='timestep', ascending=True)

#DATA FROM gSKIN
os.chdir(r"gSKIN_FILEPATH")

gskin_chest = pd.read_excel("gSKIN_chest_filename.xlsx", usecols=[0,7,8,9,13,14], names=["timestep","ACCx_uchest","ACCy_uchest","ACCz_uchest","HFa_uchest","HFb_uchest"], skiprows=14) 
gskin_chest['timestep'] = pd.to_datetime(gskin_chest['timestep'])
gskin_chest["ACCmag_uchest"] = np.sqrt(gskin_chest["ACCx_uchest"]**2 + gskin_chest["ACCy_uchest"]**2 + gskin_chest["ACCz_uchest"]**2)
gskin_chest['timestep'] = gskin_chest['timestep'].dt.floor('s')   
gskin_chest = gskin_chest.sort_values(by='timestep', ascending=True)


gskin_hand = pd.read_excel("gSKIN_hand_filename.xlsx", usecols=[0,7,8,9,12,13], names=["timestep","ACCx_hand","ACCy_hand","ACCz_hand","HFa_hand","HFb_hand"], skiprows=14) 
gskin_hand['timestep'] = pd.to_datetime(gskin_hand['timestep'])
gskin_hand["ACCmag_hand"] = np.sqrt(gskin_hand["ACCx_hand"]**2 + gskin_hand["ACCy_hand"]**2 + gskin_hand["ACCz_hand"]**2)
gskin_hand['timestep'] = gskin_hand['timestep'].dt.floor('s')
gskin_hand = gskin_hand.sort_values(by='timestep', ascending=True)



#DATA FROM EMPATICA E4
os.chdir(r"Empatica_filepath")

#The first row is the initial time of the session expressed as unix timestamp in UTC.The second row is the sample rate expressed in Hz.

#Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore the unit in this file is 1/64g. Data from x, y, and z axis are 
#respectively in first, second, and third column.
acc = pd.read_csv("ACC.csv", usecols=[0,1,2], names=["ACCx_wrist","ACCy_wrist","ACCz_wrist"])
#Data from photoplethysmograph
bvp = pd.read_csv("BVP.csv", usecols=[0], names=["BVP_wrist"])
#Data from the electrodermal activity sensor expressed as microsiemens (μS).
eda = pd.read_csv("EDA.csv", usecols=[0], names=["EDA_wrist"])
#Average heart rate extracted from the BVP signal.The first row is the initial time of the session expressed as unix timestamp in UTC.The second row is the sample rate expressed in Hz.
hr = pd.read_csv("HR.csv", usecols=[0], names=["HR_wrist"])
#Data from temperature sensor expressed degrees on the Celsius (°C) scale.
t_skin = pd.read_csv("TEMP.csv", usecols=[0], names=["Tskin_wrist"])

start_time, freq = 0, 0

#Processing the timesteps for the accelerometer data and calculating the vector magnitudes (32 hz)
start_time = datetime.utcfromtimestamp(acc["ACCx_wrist"][0]).strftime('%Y-%m-%d %H:%M:%S.%f')
freq = acc["ACCx_wrist"][1]
acc.drop([0, 1], inplace=True)
acc["timestep"] = pd.date_range(start_time, periods=acc.shape[0], freq=str(1000000/freq)+'U')
acc["ACCmag_wrist"] = np.sqrt(acc["ACCx_wrist"]**2 + acc["ACCy_wrist"]**2 + acc["ACCz_wrist"]**2)
acc = acc.sort_values(by='timestep', ascending=True)
acc = acc.set_index('timestep').resample('S').mean()

#Processing the timesteps for the bvp (64 hz)
start_time = datetime.utcfromtimestamp(bvp["BVP_wrist"][0]).strftime('%Y-%m-%d %H:%M:%S.%f')
freq = bvp["BVP_wrist"][1]
bvp.drop([0, 1], inplace=True)
bvp["timestep"] = pd.date_range(start_time, periods=bvp.shape[0], freq=str(1000000/freq)+'U')
bvp = bvp.sort_values(by='timestep', ascending=True)
bvp = bvp.set_index('timestep').resample('S').mean()

#Processing the timesteps for the eda (4 hz)
start_time = datetime.utcfromtimestamp(eda["EDA_wrist"][0]).strftime('%Y-%m-%d %H:%M:%S.%f')
freq = eda["EDA_wrist"][1]
eda.drop([0, 1], inplace=True)
eda["timestep"] = pd.date_range(start_time, periods=eda.shape[0], freq=str(1000000/freq)+'U')
eda = eda.sort_values(by='timestep', ascending=True)
eda = eda.set_index('timestep').resample('S').mean()

#Processing the timesteps for the hr (1 hz)
start_time = datetime.utcfromtimestamp(hr["HR_wrist"][0]).strftime('%Y-%m-%d %H:%M:%S.%f')
freq = hr["HR_wrist"][1]
hr.drop([0, 1], inplace=True)
hr["timestep"] = pd.date_range(start_time, periods=hr.shape[0], freq=str(1000000/freq)+'U')
hr = hr.sort_values(by='timestep', ascending=True)
hr = hr.set_index('timestep').resample('S').mean()

#Processing the timesteps for the t_wrist (1 hz)
start_time = datetime.utcfromtimestamp(t_skin["Tskin_wrist"][0]).strftime('%Y-%m-%d %H:%M:%S.%f')
freq = t_skin["Tskin_wrist"][1]
t_skin.drop([0, 1], inplace=True)
t_skin["timestep"] = pd.date_range(start_time, periods=t_skin.shape[0], freq=str(1000000/freq)+'U')
t_skin = t_skin.sort_values(by='timestep', ascending=True)
t_skin = t_skin.set_index('timestep').resample('S').mean()

#Processing the data from MOX --> NOT the good data 
os.chdir(r"MOX_filepath")
mox = pd.read_csv("filename.csv", usecols=[0,1,2,3], names=["timestep","ACCx_thigh","ACCy_thigh","ACCz_thigh"], skiprows=1)
mox['timestep'] = pd.to_datetime(mox['timestep'], format='%d-%m-%Y %H:%M:%S')
mox["ACCmag_thigh"] = np.sqrt(mox["ACCx_thigh"]**2 + mox["ACCy_thigh"]**2 + mox["ACCz_thigh"]**2)
mox = mox.sort_values(by='timestep', ascending=True)
mox = mox.set_index('timestep').resample('S').mean()

#Processing the data from Actiheart
os.chdir(r"Actiheart_filepath")
acti = pd.read_excel("Actiheart_filename.xlsx", usecols=[4], names=["BPM_chest"], skiprows=14)
start_time = datetime.strptime('YYYY-MM-DD HH:MM:SS', '%Y-%m-%d %H:%M:%S')
acti["timestep"] = pd.date_range(start_time, periods=acti.shape[0], freq='30S')

#Processing the data from Ibuttons
os.chdir(r"IButtons_filepath")
ib = pd.read_excel("iButtons_filename.xlsx", usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], skiprows=1, sheet_name='Raw Data',
                   names=["Tskin_forehead","Tskin_neck","Tskin_rscapula","Tskin_luchest","Tskin_ruarm","Tskin_luarm","Tskin_rlarm","Tskin_llarm","Tskin_lhand","Tskin_rabdomen","Tskin_rpvbl","Tskin_lpvbl","Tskin_rathigh","Tskin_lathigh",
                          "Tskin_rpthigh","Tskin_lpthigh","Tskin_rshin","Tskin_lshin","Tskin_rcalf","Tskin_lcalf","Tskin_rinstep","Tskin_linstep","Tskin_rfinger","Tskin_lfinger"])


#Processing the timesteps for the ibuttons (0.1 hz)
start_time = datetime.strptime('YYYY-MM-DD HH:MM:SS', '%Y-%m-%d %H:%M:%S')
ib["timestep"] = pd.date_range(start_time, periods=ib.shape[0], freq=str(10)+'S')
ib = ib.sort_values(by='timestep', ascending=True)

#merge all data
e4_data = pd.merge_asof(ee, eda, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, acc, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, hr, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, t_skin, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, bvp, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, mox, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, ib, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, core, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, acti, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, gskin_chest, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, gskin_hand, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, a3, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = pd.merge_asof(e4_data, b3, on = 'timestep', tolerance=pd.Timedelta('5s'))
e4_data = e4_data.ffill(axis = 0)
e4_data.dropna(inplace=True)

#drop lchest values 
e4_data = e4_data.drop(columns = ['ACCx_lchest', 'ACCy_lchest', 'ACCz_lchest','ACCmag_lchest'])
e4_data.reset_index(inplace=True)

#clean outliers in npRQ
def find_indices(list_to_check):
    return [idx for idx, value in enumerate(list_to_check) if value < 0.6 or value > 1.2]

idx_data = find_indices(e4_data.npRQ)
e4_data.loc[idx_data,'TDEE'] = np.nan
e4_data.TDEE = e4_data.TDEE.fillna(method = 'ffill')

#apply rolling mean on EE signal
e4_data.TDEE_avg = e4_data.TDEE.rolling(15).mean()


#Save the clean file 
e4_data.to_excel(r'filename.xlsx', index = False)




### Generating correlation matrix
A correlation matrix is a table showing correlation coefficients between sets of variables. Each random variable (Xi) in the table is correlated with each of the other values in the table (Xj). This allows you to see which pairs have the highest correlation.

In [ ]:
data = pd.read_excel('filename.xlsx')

corr = data.corr(method='pearson', min_periods=1)
fig_corr = px.imshow(corr, width=1800, height=1200, template="simple_white")
fig_corr.show()

### Percentage correlation explained by the variables
Often, you might be interested in seeing how much variance PCA is able to explain as you increase the number of components, in order to decide how many dimensions to ultimately keep or analyze.

In [ ]:
X = data.drop(columns=['timestep','TDEE','TDEE_avg'])
y = data[["TDEE_avg"]]

sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

pca = PCA()
pca.fit(X_std)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

fig_var = px.area(x=range(1, exp_var_cumul.shape[0] + 1),y=exp_var_cumul,labels={"x": "No. of Components", "y": "Explained Variance (-)"}, template="simple_white")
fig_var.update_xaxes(range=[0, 30])
fig_var.update_layout(width=900, height=600)
fig_var.show()

### Univariate Selection
Statistical tests can be used to select those features that have the strongest relationship with the output variable. The scikit-learn library provides the SelectKBest class that can be used with a suite of different statistical tests to select a specific number of features.

In [ ]:
# Apply SelectKBest class to extract top 10 best features
X = data.drop(columns=['timestep','TDEE','TDEE_avg']).values
y = data["TDEE_avg"].values

# feature extraction
test = SelectKBest(score_func=f_regression, k=20)
fit = test.fit(X, y)
# summarize scores
set_printoptions(precision=3)
scores = fit.scores_
print(scores)

feat_fig = go.Figure(go.Bar(x=scores, y=data.drop(columns=['timestep','TDEE','TDEE_avg']).columns, orientation='h'))
feat_fig.update_layout(width=900, height=1200, template="simple_white", xaxis_title="F-value between features and EE (-)")
feat_fig.show()

# Reduce the features to the best 20
features = fit.transform(X)